In [2]:
import pandas as pd

### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
ratings = pd.read_csv('C:\\Users\\Dmitry\\Documents\\Учеба\\3 Python и математика для анализа данных\\3.7 Pandas функции и группировки\\ml-latest-small\\ratings.csv')
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [4]:
ratings.shape

(100004, 4)

In [5]:
movies = pd.read_csv('C:\\Users\\Dmitry\\Documents\\Учеба\\3 Python и математика для анализа данных\\3.7 Pandas функции и группировки\\ml-latest-small\\movies.csv')
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [6]:
movies.shape

(9125, 3)

In [7]:
mvs_rtg = ratings.merge(movies, on='movieId', how='left')
mvs_rtg.head()[['userId', 'rating', 'title']]

,userId,rating,title
0,1,2.5,Dangerous Minds (1995)
1,1,3.0,Dumbo (1941)
2,1,3.0,Sleepers (1996)
3,1,2.0,Escape from New York (1981)
4,1,4.0,Cinema Paradiso (Nuovo cinema Paradiso) (1989)


In [8]:
mvs_rtg.shape

(100004, 6)

In [9]:
av_rate = mvs_rtg.groupby('title').mean().reset_index()[['title', 'rating']]
av_rate.head()

,title,rating
0,"""Great Performances"" Cats (1998)",1.750000
1,$9.99 (2008),3.833333
2,'Hellboy': The Seeds of Creation (2004),2.000000
3,'Neath the Arizona Skies (1934),0.500000
4,'Round Midnight (1986),2.250000


In [10]:
def classification(rating):
    if rating <= 2:
        return 'низкий рейтинг'
    elif rating <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [11]:
av_rate['class'] = av_rate['rating'].apply(classification)
av_rate.head(10)

,title,rating,class
0,"""Great Performances"" Cats (1998)",1.750000,низкий рейтинг
1,$9.99 (2008),3.833333,средний рейтинг
2,'Hellboy': The Seeds of Creation (2004),2.000000,низкий рейтинг
3,'Neath the Arizona Skies (1934),0.500000,низкий рейтинг
4,'Round Midnight (1986),2.250000,средний рейтинг
5,'Salem's Lot (2004),3.500000,средний рейтинг
6,'Til There Was You (1997),2.625000,средний рейтинг
7,"'burbs, The (1989)",3.052632,средний рейтинг
8,'night Mother (1986),5.000000,высокий рейтинг
9,(500) Days of Summer (2009),3.755556,средний рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [12]:
regions = pd.read_csv('C:\\Users\\Dmitry\\Documents\\Учеба\\3 Python и математика для анализа данных\\3.7 Pandas функции и группировки\\keywords.csv')
regions.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [13]:
def geo(keyword):
    if 'москва'in keyword or 'тула' in keyword or 'ярославль' in keyword:
        return 'Центр'
    elif 'петербург' in keyword or 'псков'in keyword or 'мурманск' in keyword:
        return 'Северо-Запад'
    elif 'владивосток'in keyword or 'сахалин' in keyword or 'хабаровск' in keyword:
        return 'Дальний Восток'
    else:
        return 'undefined'

In [14]:
regions['region'] = regions['keyword'].apply(geo)
regions['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

### Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    1. для каждой строки пройдите по всем годам списка years
    2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [15]:
av_rate.head()

,title,rating,class
0,"""Great Performances"" Cats (1998)",1.750000,низкий рейтинг
1,$9.99 (2008),3.833333,средний рейтинг
2,'Hellboy': The Seeds of Creation (2004),2.000000,низкий рейтинг
3,'Neath the Arizona Skies (1934),0.500000,низкий рейтинг
4,'Round Midnight (1986),2.250000,средний рейтинг


In [16]:
years = list(range(1950, 2011))
print(years)

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [17]:
def production_year(row):
    for year in years:
        if str(year) in row['title']:
            return year
    return 1900

In [18]:
av_rate['year'] = av_rate.apply(production_year, axis=1)
av_rate.head()

,title,rating,class,year
0,"""Great Performances"" Cats (1998)",1.750000,низкий рейтинг,1998
1,$9.99 (2008),3.833333,средний рейтинг,2008
2,'Hellboy': The Seeds of Creation (2004),2.000000,низкий рейтинг,2004
3,'Neath the Arizona Skies (1934),0.500000,низкий рейтинг,1900
4,'Round Midnight (1986),2.250000,средний рейтинг,1986


In [19]:
av_rate.groupby('year').mean().reset_index().sort_values('rating', ascending = False).head(10)

,year,rating
2,1951,3.906653
3,1952,3.802318
13,1962,3.772962
4,1953,3.738878
8,1957,3.693322
24,1973,3.688212
22,1971,3.643227
20,1969,3.604509
23,1972,3.585295
9,1958,3.571956
